In [1]:
import glob, os 
import pandas as pd
import numpy as np

In [93]:
def cleaning (df):
    df.columns = df.columns.to_series().apply(lambda x: x.strip())
    df.columns = df.columns.str.replace(' ', '_')
    df.columns = df.columns.str.replace('-', '_')

def observations (df):
    df['Demo_ID'] = df['Demo_ID'].astype(str)
    df['Subdealers_Dealers_Farmers_Villages_Id'] = df['Subdealers_Dealers_Farmers_Villages_Id'].astype(str)
    df['DemoID_OutletID'] = df['Demo_ID'] + df['Subdealers_Dealers_Farmers_Villages_Id']
    
def droptest(df):
    df.drop(df[df['User_Id'] == 8].index, inplace = True) 
    df.drop(df[df['User_Id'] == 4].index, inplace = True) 
    df.drop(df[df['User_Id'] == 7].index, inplace = True) 

In [100]:
path =r'D:\Bizom\Bizom Analysis\Demo Analysis\DemoFY21-22\Demo Reports'
demo = pd.concat(map(pd.read_excel, glob.glob(os.path.join(path, "*.xlsx"))),sort=False)

In [101]:
cleaning (demo)

demo['Demo_Id'] = demo['Demo_Id'].astype(str)
demo['Outlet_Id'] = demo['Outlet_Id'].astype(str)
demo['con']= demo['Demo_Id']+ demo['Outlet_Id']

# All level observation of all crops
cot_demo = demo.loc[demo['Focus_Crop'] == "Cotton"].copy()
paddy_demo = demo.loc[demo['Focus_Crop'] == "Paddy"].copy()
scane_demo = demo.loc[demo['Focus_Crop'] == "Sugarcane"].copy()
soy_demo = demo.loc[demo['Focus_Crop'] == "Soybean"].copy()
gnut_demo = demo.loc[demo['Focus_Crop'] == "Groundnut"].copy()
tom_demo = demo.loc[demo['Focus_Crop'] == "Tomato"].copy()
maize_demo = demo.loc[demo['Focus_Crop'] == "Maize"].copy()
oni_demo = demo.loc[demo['Focus_Crop'] == "Onion"].copy()
pot_demo = demo.loc[demo['Focus_Crop'] == "Potato"].copy()
banana_demo = demo.loc[demo['Focus_Crop'] == "Banana"].copy()
must_demo = demo.loc[demo['Focus_Crop'] == "Mustard"].copy()
wheat_demo = demo.loc[demo['Focus_Crop'] == "Wheat"].copy()

In [102]:
file = r'D:\Bizom\Bizom Analysis\Demo Analysis\DemoFY21-22\Outlet_Dump\dump.csv'

outlet = pd.read_csv(file)

C:\Users\ritesh.gane\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (7,12,14,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [103]:
fd = outlet.loc[outlet['Outlet_Type'] == "Farmer"].copy()
fdn = fd[['Bizom_Id','Taluka_Name', 'District' , 'State']].copy()
fdn['Bizom_Id'] = fdn['Bizom_Id'].astype(str)

In [104]:
demo1 = demo.merge(fdn, left_on = 'Outlet_Id', right_on = 'Bizom_Id', how = 'left')
demo1.shape

(8895, 38)

# Cotton Observations

In [7]:
path =r'D:\Bizom\Bizom Analysis\Demo Analysis\DemoFY21-22\Crop_Observations\Cotton'
cot_ob = pd.concat(map(pd.read_excel, glob.glob(os.path.join(path, "*.xlsx"))),sort=False)

cleaning (cot_ob)
observations(cot_ob)
droptest(cot_ob)

cot_f = cot_demo.merge(cot_ob,left_on= 'con', right_on = 'DemoID_OutletID')
cot_f.shape

(560, 74)

In [8]:
cot_ob_nm = cot_ob.merge(cot_f,left_on= 'DemoID_OutletID', right_on = 'DemoID_OutletID', how='outer',indicator=True)
cot_ob_nm=cot_ob_nm[cot_ob_nm['_merge']=='left_only']
cot_ob_nm.shape

(81, 114)

In [9]:
cot_f['con_demo_ID_Observation'] = cot_f['Demo_Id']+ cot_f['Observation_No_Cotton']
cot_fl = cot_f.drop_duplicates(subset ="con_demo_ID_Observation", keep='last')

cot_fl1= pd.pivot_table(cot_fl,values ='Name', columns ='Observation_No_Cotton',
                        index = ['Demo_ID'], aggfunc={'Name':'count'})

cot_fl2 = pd.DataFrame(cot_fl1.to_records())
cleaning (cot_fl2)
cot_fl2.fillna(0, inplace = True)
cot_fl2.columns

Index(['Demo_ID', '120_Days_After_Planting', '60_Days_After_Planting',
       '90_Days_After_Planting', 'At_Harvest'],
      dtype='object')

In [10]:

cot_fl2['60_Days_After_Planting_v1'] = cot_fl2.apply(lambda row: 1.0 if ((row['60_Days_After_Planting']) == 1) and
                                    ((row['90_Days_After_Planting']) < 1) and ((row['120_Days_After_Planting']) < 1) and 
                                            ((row['At_Harvest']) <1)
                                        else 0 ,axis=1) 


cot_fl2['90_Days_After_Planting_v1'] = cot_fl2.apply(lambda row: 1.0 if ((row['60_Days_After_Planting']) == 1) and
                                    ((row['90_Days_After_Planting']) == 1) and ((row['120_Days_After_Planting']) < 1) and 
                                             ((row['At_Harvest']) <1)
                                     else 0 ,axis=1) 


cot_fl2['120_Days_After_Planting_v1'] = cot_fl2.apply(lambda row: 1.0 if ((row['60_Days_After_Planting']) == 1) and
                                    ((row['90_Days_After_Planting']) == 1) and ((row['120_Days_After_Planting']) == 1) and 
                                               ((row['At_Harvest']) <1)
                                     else 0 ,axis=1) 


cot_fl2['At_Harvest_v1'] = cot_fl2.apply(lambda row: 1.0 if ((row['60_Days_After_Planting']) == 1) and
                                     ((row['90_Days_After_Planting']) == 1) and ((row['120_Days_After_Planting']) == 1) and 
                                                  ((row['At_Harvest']) ==1)
                                        else 0 ,axis=1) 


cot_fl3=  pd.pivot_table(cot_fl2,values =['60_Days_After_Planting_v1','90_Days_After_Planting_v1',
                                         '120_Days_After_Planting_v1','At_Harvest_v1'],
                         index = ['Demo_ID'], aggfunc=np.sum)

cot_fl4= pd.DataFrame(cot_fl3.to_records())
cot_fl4.head()

,Demo_ID,120_Days_After_Planting_v1,60_Days_After_Planting_v1,90_Days_After_Planting_v1,At_Harvest_v1
0,1007965,0,0.0,0.0,0.0
1,1008101,0,1.0,0.0,0.0
2,1008150,0,1.0,0.0,0.0
3,1008212,0,1.0,0.0,0.0
4,1008503,0,1.0,0.0,0.0


# Onion Observations

In [11]:
path =r'D:\Bizom\Bizom Analysis\Demo Analysis\DemoFY21-22\Crop_Observations\Oni'
onion_ob = pd.concat(map(pd.read_excel, glob.glob(os.path.join(path, "*.xlsx"))),sort=False)

cleaning (onion_ob)
droptest(onion_ob)
observations(onion_ob)

onion_f = oni_demo.merge(onion_ob,left_on= 'con', right_on = 'DemoID_OutletID')
onion_f.shape


(5, 72)

In [12]:
oni_ob_nm = onion_ob.merge(onion_f,left_on= 'DemoID_OutletID', right_on = 'DemoID_OutletID', how='outer',indicator=True)
oni_ob_nm=oni_ob_nm[oni_ob_nm['_merge']=='left_only']
oni_ob_nm.shape

(2320, 110)

In [13]:
onion_f['con_demo_ID_Observation'] = onion_f['Demo_Id']+ onion_f['Observation_No_Onion']
onion_fl = onion_f.drop_duplicates(subset ="con_demo_ID_Observation", keep='last')


onion_fl1= pd.pivot_table(onion_fl,values ='Name', columns ='Observation_No_Onion',
                        index = ['Demo_ID'], aggfunc={'Name':'count'})

oni_fl2 = pd.DataFrame(onion_fl1.to_records())
cleaning (oni_fl2)
oni_fl2.fillna(0, inplace = True)
oni_fl2.columns


Index(['Demo_ID', '30_Days_After_Planting', '60_Days_After_Planting',
       '90_Days_After_Planting', 'At_Harvest'],
      dtype='object')

In [14]:
oni_fl2['30_Days_After_Planting_v1'] = oni_fl2.apply(lambda row: 1.0 if ((row['30_Days_After_Planting']) == 1) and
                                                     ((row['60_Days_After_Planting']) < 1) and
                                                     ((row['90_Days_After_Planting']) < 1) and
                                                     ((row['At_Harvest']) <1) else 0 ,axis=1) 

oni_fl2['60_Days_After_Planting_v1'] = oni_fl2.apply(lambda row: 1.0 if ((row['30_Days_After_Planting']) == 1) and
                                                     ((row['60_Days_After_Planting']) == 1) and
                                                     ((row['90_Days_After_Planting']) < 1) and
                                                     ((row['At_Harvest']) <1) else 0 ,axis=1) 

oni_fl2['90_Days_After_Planting_v1'] = oni_fl2.apply(lambda row: 1.0 if ((row['30_Days_After_Planting']) == 1) and
                                                     ((row['60_Days_After_Planting']) == 1) and
                                                     ((row['90_Days_After_Planting']) == 1) and
                                                     ((row['At_Harvest']) <1) else 0 ,axis=1) 

oni_fl2['At_Harvest_v1'] = oni_fl2.apply(lambda row: 1.0 if ((row['30_Days_After_Planting']) == 1) and
                                                     ((row['60_Days_After_Planting']) == 1) and
                                                     ((row['90_Days_After_Planting']) == 1) and   
                                                     ((row['At_Harvest']) ==1) else 0 ,axis=1) 

oni_fl3=  pd.pivot_table(oni_fl2,values =['30_Days_After_Planting_v1','60_Days_After_Planting_v1',
                                         '90_Days_After_Planting_v1', 'At_Harvest_v1'], 
                                            index = ['Demo_ID'], aggfunc=np.sum)

oni_fl4= pd.DataFrame(oni_fl3.to_records())
oni_fl4.head()


,Demo_ID,30_Days_After_Planting_v1,60_Days_After_Planting_v1,90_Days_After_Planting_v1,At_Harvest_v1
0,97634298411,0,0,0,1.0


# Tomato Observations

In [15]:
path =r'D:\Bizom\Bizom Analysis\Demo Analysis\DemoFY21-22\Crop_Observations\tom'
tom_ob = pd.concat(map(pd.read_excel, glob.glob(os.path.join(path, "*.xlsx"))),sort=False)

cleaning (tom_ob)
observations(tom_ob)
droptest(tom_ob)

tom_f = tom_demo.merge(tom_ob,left_on= 'con', right_on = 'DemoID_OutletID')
tom_f.shape


(34, 76)

In [16]:
tom_ob_nm = tom_ob.merge(tom_f,left_on= 'DemoID_OutletID', right_on = 'DemoID_OutletID', how='outer',indicator=True)
tom_ob_nm=tom_ob_nm[tom_ob_nm['_merge']=='left_only']
tom_ob_nm.shape

(94, 118)

In [17]:
tom_f['con_demo_ID_Observation'] = tom_f['Demo_Id']+ tom_f['Observation_No_Tomato']
tom_fl = tom_f.drop_duplicates(subset ="con_demo_ID_Observation", keep='last')

tom_fl1= pd.pivot_table(tom_fl,values ='Name', columns ='Observation_No_Tomato',
                          index = ['Demo_ID'],  aggfunc={'Name':'count'})

tom_fl2 = pd.DataFrame(tom_fl1.to_records())
cleaning(tom_fl2)
tom_fl2.fillna(0, inplace = True)
tom_fl2.columns

Index(['Demo_ID', '30_Days_After_Planting', '60_Days_After_Planting',
       '90_Days_After_Planting', 'At_Harvest'],
      dtype='object')

In [18]:
tom_fl2['30_Days_After_Planting_v1'] = tom_fl2.apply(lambda row: 1.0 if ((row['30_Days_After_Planting']) == 1) and
                                                     ((row['60_Days_After_Planting']) < 1) and
                                                     ((row['90_Days_After_Planting']) < 1) and
                                                     ((row['At_Harvest']) <1) else 0 ,axis=1)

tom_fl2['60_Days_After_Planting_v1'] = tom_fl2.apply(lambda row: 1.0 if ((row['30_Days_After_Planting']) == 1) and
                                                     ((row['60_Days_After_Planting']) == 1) and
                                                     ((row['90_Days_After_Planting']) < 1) and
                                                     ((row['At_Harvest']) <1) else 0 ,axis=1) 

tom_fl2['90_Days_After_Planting_v1'] = tom_fl2.apply(lambda row: 1.0 if ((row['30_Days_After_Planting']) == 1) and
                                                     ((row['60_Days_After_Planting']) == 1) and
                                                     ((row['90_Days_After_Planting']) == 1) and
                                                     ((row['At_Harvest']) <1) else 0 ,axis=1) 

tom_fl2['At_Harvest_v1'] = tom_fl2.apply(lambda row: 1.0 if ((row['30_Days_After_Planting']) == 1) and
                                                     ((row['60_Days_After_Planting']) == 1) and
                                                     ((row['90_Days_After_Planting']) == 1) and   
                                                     ((row['At_Harvest']) ==1) else 0 ,axis=1)  


tom_fl3=  pd.pivot_table(tom_fl2,values =['30_Days_After_Planting_v1','60_Days_After_Planting_v1','90_Days_After_Planting_v1',
                                         'At_Harvest_v1'], index = ['Demo_ID'], aggfunc=np.sum)

tom_fl4= pd.DataFrame(tom_fl3.to_records())
tom_fl4.head()

,Demo_ID,30_Days_After_Planting_v1,60_Days_After_Planting_v1,90_Days_After_Planting_v1,At_Harvest_v1
0,1064439,1.0,0.0,0,0.0
1,1064443,1.0,0.0,0,0.0
2,1064454,1.0,0.0,0,0.0
3,1067817,1.0,0.0,0,0.0
4,1067842,1.0,0.0,0,0.0


# Soybean Observations

In [19]:
path =r'D:\Bizom\Bizom Analysis\Demo Analysis\DemoFY21-22\Crop_Observations\soy'
soy_ob = pd.concat(map(pd.read_excel, glob.glob(os.path.join(path, "*.xlsx"))),sort=False)

cleaning (soy_ob)
observations(soy_ob)
droptest(soy_ob)

soy_f = soy_demo.merge(soy_ob,left_on= 'con', right_on = 'DemoID_OutletID')
soy_f.shape

(889, 67)

In [20]:
soy_ob_nm = soy_ob.merge(soy_f,left_on= 'DemoID_OutletID', right_on = 'DemoID_OutletID', how='outer',indicator=True)
soy_ob_nm=soy_ob_nm[soy_ob_nm['_merge']=='left_only']
soy_ob_nm.shape

(283, 100)

In [21]:
soy_f['con_demo_ID_Observation'] = soy_f['Demo_Id']+ soy_f['Demo_Crop__Soybean']
soy_fl = soy_f.drop_duplicates(subset ="con_demo_ID_Observation", keep='last')

soy_fl1= pd.pivot_table(soy_fl,values ='Name', columns ='Demo_Crop__Soybean',
                        index = ['Demo_ID'], aggfunc={'Name':'count'})

soy_fl2 = pd.DataFrame(soy_fl1.to_records())

soy_fl2.fillna(0, inplace = True)
cleaning(soy_fl2)
soy_fl2.columns


Index(['Demo_ID', '30_Days_After_Planting', '90_Days_After_Planting',
       'At_Harvest'],
      dtype='object')

In [61]:
soy_fl2['30_Days_After_Planting_v1'] = soy_fl2.apply(lambda row: 1.0 if ((row['30_Days_After_Planting']) == 1) and
                                     ((row['90_Days_After_Planting']) < 1) and
                                    ((row['At_Harvest']) <1)
                                        else 0 ,axis=1) 

soy_fl2['90_Days_After_Planting_v1'] = soy_fl2.apply(lambda row: 1.0 if ((row['30_Days_After_Planting']) == 1) and
                                     ((row['90_Days_After_Planting']) == 1) and ((row['At_Harvest']) <1)
                                        else 0 ,axis=1) 

soy_fl2['At_Harvest_v1'] = soy_fl2.apply(lambda row: 1.0 if ((row['30_Days_After_Planting']) == 1) and
                                     ((row['90_Days_After_Planting']) == 1) and ((row['At_Harvest']) == 1)
                                        else 0 ,axis=1) 

soy_fl3=  pd.pivot_table(soy_fl2,values =['30_Days_After_Planting_v1','90_Days_After_Planting_v1','At_Harvest_v1'],
                         index = ['Demo_ID'], aggfunc=np.sum)

soy_fl4= pd.DataFrame(soy_fl3.to_records())
soy_fl4.head()


,Demo_ID,30_Days_After_Planting_v1,90_Days_After_Planting_v1,At_Harvest_v1
0,1003443,0.0,1.0,0.0
1,1003522,0.0,0.0,0.0
2,1003593,0.0,1.0,0.0
3,1003658,1.0,0.0,0.0
4,1003688,0.0,0.0,0.0


# Groundnut Observations

In [23]:
path =r'D:\Bizom\Bizom Analysis\Demo Analysis\DemoFY21-22\Crop_Observations\gnut'
gnut_ob = pd.concat(map(pd.read_excel, glob.glob(os.path.join(path, "*.xlsx"))),sort=False)

cleaning (gnut_ob)
observations(gnut_ob)
droptest(gnut_ob)

gnut_f = gnut_demo.merge(gnut_ob,left_on= 'con', right_on = 'DemoID_OutletID')
gnut_f.shape

(2, 59)

In [24]:
gnut_ob_nm = gnut_ob.merge(gnut_f,left_on= 'DemoID_OutletID', right_on = 'DemoID_OutletID', how='outer',indicator=True)
gnut_ob_nm=gnut_ob_nm[gnut_ob_nm['_merge']=='left_only']
gnut_ob_nm.shape

(420, 84)

In [25]:
gnut_f['con_demo_ID_Observation'] = gnut_f['Demo_Id']+ gnut_f['Demo_Crop__Groundnut']
gnut_fl = gnut_f.drop_duplicates(subset ="con_demo_ID_Observation", keep='last')

gnut_fl1= pd.pivot_table(gnut_fl,values ='Name', columns ='Demo_Crop__Groundnut',
                          index = ['Demo_ID'],  aggfunc={'Name':'count'})

gnut_fl2 = pd.DataFrame(gnut_fl1.to_records())
gnut_fl2.fillna(0, inplace = True)
cleaning(gnut_fl2)
gnut_fl2.columns

Index(['Demo_ID', '60_Days_After_Planting', 'At_Harvest'], dtype='object')

In [26]:
gnut_fl2['60_Days_After_Planting_v1'] = gnut_fl2.apply(lambda row: 1.0 if ((row['60_Days_After_Planting']) == 1) 
                                                       and((row['At_Harvest']) <1) else 0 ,axis=1) 

gnut_fl2['At_Harvest_v1'] = gnut_fl2.apply(lambda row: 1.0 if ((row['60_Days_After_Planting']) == 1) and
                                        ((row['At_Harvest']) == 1) else 0 ,axis=1) 


gnut_fl3=  pd.pivot_table(gnut_fl2,values =['60_Days_After_Planting_v1','At_Harvest_v1'],index = ['Demo_ID'], aggfunc=np.sum)


gnut_fl4= pd.DataFrame(gnut_fl3.to_records())
gnut_fl4.head()

,Demo_ID,60_Days_After_Planting_v1,At_Harvest_v1
0,97634298415,0,1.0


# Sugarcane Observations

In [27]:
path =r'D:\Bizom\Bizom Analysis\Demo Analysis\DemoFY21-22\Crop_Observations\scane'
scane_ob = pd.concat(map(pd.read_excel, glob.glob(os.path.join(path, "*.xlsx"))),sort=False)

cleaning (scane_ob)
observations(scane_ob)
droptest(scane_ob)

scane_f = scane_demo.merge(scane_ob,left_on= 'con', right_on = 'DemoID_OutletID')
scane_f.shape

(23, 74)

In [28]:
scane_ob_nm = scane_ob.merge(scane_f,left_on= 'DemoID_OutletID', right_on = 'DemoID_OutletID', how='outer',indicator=True)
scane_ob_nm=scane_ob_nm[scane_ob_nm['_merge']=='left_only']
scane_ob_nm.shape

(1669, 114)

In [29]:
scane_f['con_demo_ID_Observation'] = scane_f['Demo_Id']+ scane_f['Observation_No_Sugarcane']
scane_fl = scane_f.drop_duplicates(subset ="con_demo_ID_Observation", keep='last')

scane_fl1= pd.pivot_table(scane_fl,values ='Name', columns ='Observation_No_Sugarcane',
                          index = ['Demo_ID'],  aggfunc={'Name':'count'})

scane_fl2 = pd.DataFrame(scane_fl1.to_records())
scane_fl2.fillna(0, inplace = True)
cleaning(scane_fl2)
scane_fl2.columns

Index(['Demo_ID', '.', '120_Days_After_Planting', '150_Days_After_Planting',
       '90_Days_After_Planting', 'At_Harvest'],
      dtype='object')

In [30]:
scane_fl2['90_Days_After_Planting_v1'] = scane_fl2.apply(lambda row: 1.0 if ((row['90_Days_After_Planting']) == 1) and
                                         ((row['120_Days_After_Planting']) <1) and
                                         ((row['150_Days_After_Planting']) <1) else 0 ,axis=1) 

scane_fl2['120_Days_After_Planting_v1'] = scane_fl2.apply(lambda row: 1.0 if ((row['90_Days_After_Planting']) == 1) and
                                         ((row['120_Days_After_Planting']) == 1) and
                                         ((row['150_Days_After_Planting']) <1) else 0 ,axis=1) 

scane_fl2['150_Days_After_Planting_v1'] = scane_fl2.apply(lambda row: 1.0 if ((row['90_Days_After_Planting']) == 1) and
                                         ((row['120_Days_After_Planting']) == 1) and
                                         ((row['150_Days_After_Planting']) == 1)  else 0 ,axis=1) 

scane_fl3=  pd.pivot_table(scane_fl2,values =['90_Days_After_Planting_v1','120_Days_After_Planting_v1', 
                                              '150_Days_After_Planting_v1' ], index = ['Demo_ID'], aggfunc=np.sum)

scane_fl4= pd.DataFrame(scane_fl3.to_records())
scane_fl4.head()

,Demo_ID,120_Days_After_Planting_v1,150_Days_After_Planting_v1,90_Days_After_Planting_v1
0,1049581,0,0.0,0.0
1,1050068,0,0.0,0.0
2,1070639,0,0.0,0.0
3,1070691,0,0.0,0.0
4,1076557,0,0.0,1.0


# Paddy Observations

In [31]:
path =r'D:\Bizom\Bizom Analysis\Demo Analysis\DemoFY21-22\Crop_Observations\paddy'
paddy_ob = pd.concat(map(pd.read_excel, glob.glob(os.path.join(path, "*.xlsx"))),sort=False)

cleaning (paddy_ob)
observations(paddy_ob)
droptest(paddy_ob)

paddy_f = paddy_demo.merge(paddy_ob,left_on= 'con', right_on = 'DemoID_OutletID')
paddy_f.shape

(13, 59)

In [32]:
paddy_ob_nm = paddy_ob.merge(paddy_f,left_on= 'DemoID_OutletID', right_on = 'DemoID_OutletID', how='outer',indicator=True)
paddy_ob_nm=paddy_ob_nm[paddy_ob_nm['_merge']=='left_only']
paddy_ob_nm.shape

(198, 84)

In [33]:
paddy_f['con_demo_ID_Observation'] = paddy_f['Demo_Id']+ paddy_f['Demo_Crop___Paddy']
paddy_fl = paddy_f.drop_duplicates(subset ="con_demo_ID_Observation", keep='last')

paddy_fl1= pd.pivot_table(paddy_fl,values ='Name', columns ='Demo_Crop___Paddy',
                          index = ['Demo_ID'],aggfunc={'Name':'count'})

paddy_fl2 = pd.DataFrame(paddy_fl1.to_records())

paddy_fl2.fillna(0, inplace = True)
cleaning(paddy_fl2)
paddy_fl2.columns

Index(['Demo_ID', '60_Days_After_Planting', 'At_Harvest'], dtype='object')

In [60]:
paddy_fl2['60_Days_After_Planting_v1'] = paddy_fl2.apply(lambda row: 1.0 if ((row['60_Days_After_Planting']) == 1) and
                                        ((row['At_Harvest']) < 1) else 0 ,axis=1) 

paddy_fl2['At_Harvest_v1'] = paddy_fl2.apply(lambda row: 1.0 if ((row['60_Days_After_Planting']) == 1) and
                                        ((row['At_Harvest']) ==1) else 0 ,axis=1) 


paddy_fl3=  pd.pivot_table(paddy_fl2,values =['60_Days_After_Planting_v1','At_Harvest_v1'],index = ['Demo_ID'], aggfunc=np.sum)
paddy_fl4= pd.DataFrame(paddy_fl3.to_records())
paddy_fl4.head()

,Demo_ID,60_Days_After_Planting_v1,At_Harvest_v1
0,1188769,1.0,0.0
1,1190575,1.0,0.0
2,1237289,1.0,0.0
3,1237465,1.0,0.0
4,1237596,1.0,0.0


# Maize Observations

In [35]:
path =r'D:\Bizom\Bizom Analysis\Demo Analysis\DemoFY21-22\Crop_Observations\maize'
maize_ob = pd.concat(map(pd.read_excel, glob.glob(os.path.join(path, "*.xlsx"))),sort=False)

cleaning (maize_ob)
observations(maize_ob)
droptest(maize_ob)

maize_f = maize_demo.merge(maize_ob,left_on= 'con', right_on = 'DemoID_OutletID')
maize_f.shape

(83, 66)

In [36]:
maize_ob_nm = maize_ob.merge(maize_f,left_on= 'DemoID_OutletID', right_on = 'DemoID_OutletID', how='outer',indicator=True)
maize_ob_nm=maize_ob_nm[maize_ob_nm['_merge']=='left_only']
maize_ob_nm.shape

(43, 98)

In [37]:
maize_f['con_demo_ID_Observation'] = maize_f['Demo_Id']+ maize_f['Observation_No_Maize']
maize_fl = maize_f.drop_duplicates(subset ="con_demo_ID_Observation", keep='last')

maize_fl1= pd.pivot_table(maize_fl,values ='Name', columns ='Observation_No_Maize',
                          index = ['Demo_ID'],  aggfunc={'Name':'count'})

maize_fl2 = pd.DataFrame(maize_fl1.to_records())

maize_fl2.fillna(0, inplace = True)
cleaning(maize_fl2)
maize_fl2.columns

Index(['Demo_ID', '30_Days_After_Planting', '60_Days_After_Planting',
       'At_Harvest'],
      dtype='object')

In [38]:
maize_fl2['30_Days_After_Planting_v1'] = maize_fl2.apply(lambda row: 1.0 if ((row['30_Days_After_Planting']) == 1) and
                                                     ((row['60_Days_After_Planting']) < 1)and
                                                     ((row['At_Harvest']) < 1)
                                                         else 0 ,axis=1) 

maize_fl2['60_Days_After_Planting_v1'] = maize_fl2.apply(lambda row: 1.0 if ((row['30_Days_After_Planting']) == 1) and
                                                     ((row['60_Days_After_Planting']) == 1)and
                                                     ((row['At_Harvest']) <1)  else 0 ,axis=1) 

maize_fl2['At_Harvest_v1'] = maize_fl2.apply(lambda row: 1.0 if ((row['30_Days_After_Planting']) == 1) and
                                                   ((row['60_Days_After_Planting']) == 1) and
                                                     ((row['At_Harvest']) ==1) else 0 ,axis=1) 
maize_fl2['Crop'] = 'maize'
maize_fl3=  pd.pivot_table(maize_fl2,values =['30_Days_After_Planting_v1','60_Days_After_Planting_v1','At_Harvest_v1'], 
                                            index = ['Demo_ID'], aggfunc=np.sum)

maize_fl4= pd.DataFrame(maize_fl3.to_records())
maize_fl4.head()

,Demo_ID,30_Days_After_Planting_v1,60_Days_After_Planting_v1,At_Harvest_v1
0,1049550,0,1.0,0.0
1,1060685,0,1.0,0.0
2,1060687,0,1.0,0.0
3,1061534,0,1.0,0.0
4,1061536,0,1.0,0.0


# Mustard Observations

In [39]:
path =r'D:\Bizom\Bizom Analysis\Demo Analysis\DemoFY21-22\Crop_Observations\mustard'
must_ob = pd.concat(map(pd.read_excel, glob.glob(os.path.join(path, "*.xlsx"))),sort=False)

cleaning (must_ob)
observations(must_ob)
droptest(must_ob)

must_f = must_demo.merge(must_ob,left_on= 'con', right_on = 'DemoID_OutletID')
must_f.shape

(5, 69)

In [40]:
must_ob_nm = must_ob.merge(must_f,left_on= 'DemoID_OutletID', right_on = 'DemoID_OutletID', how='outer',indicator=True)
must_ob_nm=must_ob_nm[must_ob_nm['_merge']=='left_only']
must_ob_nm.shape

(0, 104)

In [41]:
must_f['con_demo_ID_Observation'] = must_f['Demo_Id']+ must_f['Demo_Crop___Mustard']
must_fl = must_f.drop_duplicates(subset ="con_demo_ID_Observation", keep='last')

must_fl1= pd.pivot_table(must_fl,values ='Name', columns ='Demo_Crop___Mustard',
                          index = ['Demo_ID'],  aggfunc={'Name':'count'})

must_fl2 = pd.DataFrame(must_fl1.to_records())

must_fl2.fillna(0, inplace = True)
cleaning(must_fl2)
must_fl2.columns

Index(['Demo_ID', '30_Days_After_Planting', '60_Days_After_Planting',
       '90_Days_After_Planting', 'At_Harvest'],
      dtype='object')

In [43]:
must_fl2['30_Days_After_Planting_v1'] = must_fl2.apply(lambda row: 1.0 if ((row['30_Days_After_Planting']) == 1) and
                                                     ((row['60_Days_After_Planting']) < 1) and
                                                     ((row['90_Days_After_Planting']) < 1) and
                                                     ((row['At_Harvest']) <1) else 0 ,axis=1)

must_fl2['60_Days_After_Planting_v1'] = must_fl2.apply(lambda row: 1.0 if ((row['30_Days_After_Planting']) == 1) and
                                                     ((row['60_Days_After_Planting']) == 1) and
                                                     ((row['90_Days_After_Planting']) < 1) and
                                                     ((row['At_Harvest']) <1) else 0 ,axis=1) 

must_fl2['90_Days_After_Planting_v1'] = must_fl2.apply(lambda row: 1.0 if ((row['30_Days_After_Planting']) == 1) and
                                                     ((row['60_Days_After_Planting']) == 1) and
                                                     ((row['90_Days_After_Planting']) == 1) and
                                                     ((row['At_Harvest']) <1) else 0 ,axis=1) 

must_fl2['At_Harvest_v1'] = must_fl2.apply(lambda row: 1.0 if ((row['30_Days_After_Planting']) == 1) and
                                                     ((row['60_Days_After_Planting']) == 1) and
                                                     ((row['90_Days_After_Planting']) == 1) and   
                                                     ((row['At_Harvest']) ==1) else 0 ,axis=1)  


must_fl3=  pd.pivot_table(must_fl2,values =['30_Days_After_Planting_v1','60_Days_After_Planting_v1','90_Days_After_Planting_v1',
                                         'At_Harvest_v1'], index = ['Demo_ID'], aggfunc=np.sum)

must_fl4= pd.DataFrame(must_fl3.to_records())
must_fl4.head()

,Demo_ID,30_Days_After_Planting_v1,60_Days_After_Planting_v1,90_Days_After_Planting_v1,At_Harvest_v1
0,97634298420,0,0,0,1.0


# Banana Observations

In [44]:
path =r'D:\Bizom\Bizom Analysis\Demo Analysis\DemoFY21-22\Crop_Observations\banana'
banana_ob = pd.concat(map(pd.read_excel, glob.glob(os.path.join(path, "*.xlsx"))),sort=False)

cleaning (banana_ob)
observations(banana_ob)
droptest(banana_ob)

banana_f = banana_demo.merge(banana_ob,left_on= 'con', right_on = 'DemoID_OutletID')
banana_f.shape

(6, 83)

In [45]:
banana_ob_nm = banana_ob.merge(banana_f,left_on= 'DemoID_OutletID', right_on = 'DemoID_OutletID', how='outer',indicator=True)
banana_ob_nm=banana_ob_nm[banana_ob_nm['_merge']=='left_only']
banana_ob_nm.shape

(0, 132)

In [46]:
banana_f['con_demo_ID_Observation'] = banana_f['Demo_Id']+ banana_f['Demo_Crop___Banana']
banana_fl = banana_f.drop_duplicates(subset ="con_demo_ID_Observation", keep='last')

banana_fl1= pd.pivot_table(banana_fl,values ='Name', columns ='Demo_Crop___Banana',
                          index = ['Demo_ID'],  aggfunc={'Name':'count'})

banana_fl2 = pd.DataFrame(banana_fl1.to_records())

banana_fl2.fillna(0, inplace = True)
cleaning(banana_fl2)
banana_fl2.columns

Index(['Demo_ID', '150_Days_After_Planting', '180_Days_After_Planting',
       '210_Days_After_Planting', '240_Days_After_Planting',
       '270_Days_After_Planting', 'At_Harvest'],
      dtype='object')

In [47]:
banana_fl2['150_Days_After_Planting_v1'] = banana_fl2.apply(lambda row: 1.0 if ((row['150_Days_After_Planting']) == 1) and
                                                     ((row['180_Days_After_Planting']) < 1) and
                                                     ((row['210_Days_After_Planting']) < 1) and
                                                     ((row['240_Days_After_Planting']) < 1) and
                                                     ((row['270_Days_After_Planting']) < 1) and
                                                     ((row['At_Harvest']) < 1) else 0 ,axis=1)

banana_fl2['180_Days_After_Planting_v1'] = banana_fl2.apply(lambda row: 1.0 if ((row['150_Days_After_Planting']) == 1) and
                                                     ((row['180_Days_After_Planting']) == 1) and
                                                     ((row['210_Days_After_Planting']) < 1) and
                                                     ((row['240_Days_After_Planting']) < 1) and
                                                     ((row['270_Days_After_Planting']) < 1) and
                                                     ((row['At_Harvest']) < 1) else 0 ,axis=1)

banana_fl2['210_Days_After_Planting_v1'] = banana_fl2.apply(lambda row: 1.0 if ((row['150_Days_After_Planting']) == 1) and
                                                     ((row['180_Days_After_Planting']) == 1) and
                                                     ((row['210_Days_After_Planting']) == 1) and
                                                     ((row['240_Days_After_Planting']) < 1) and
                                                     ((row['270_Days_After_Planting']) < 1) and
                                                     ((row['At_Harvest']) < 1) else 0 ,axis=1)

banana_fl2['240_Days_After_Planting_v1'] = banana_fl2.apply(lambda row: 1.0 if ((row['150_Days_After_Planting']) == 1) and
                                                     ((row['180_Days_After_Planting']) == 1) and
                                                     ((row['210_Days_After_Planting']) == 1) and
                                                     ((row['240_Days_After_Planting']) == 1) and
                                                     ((row['270_Days_After_Planting']) < 1) and
                                                     ((row['At_Harvest']) < 1) else 0 ,axis=1) 

banana_fl2['270_Days_After_Planting_v1'] = banana_fl2.apply(lambda row: 1.0 if ((row['150_Days_After_Planting']) == 1) and
                                                     ((row['180_Days_After_Planting']) == 1) and
                                                     ((row['210_Days_After_Planting']) == 1) and
                                                     ((row['240_Days_After_Planting']) == 1) and
                                                     ((row['270_Days_After_Planting']) == 1) and
                                                     ((row['At_Harvest']) < 1) else 0 ,axis=1) 

banana_fl2['At_Harvest_v1'] = banana_fl2.apply(lambda row: 1.0 if ((row['150_Days_After_Planting']) == 1) and
                                                     ((row['180_Days_After_Planting']) == 1) and
                                                     ((row['210_Days_After_Planting']) == 1) and
                                                     ((row['240_Days_After_Planting']) == 1) and
                                                     ((row['270_Days_After_Planting']) == 1) and
                                                     ((row['At_Harvest']) == 1) else 0 ,axis=1) 


banana_fl3=  pd.pivot_table(banana_fl2,values =['150_Days_After_Planting_v1','180_Days_After_Planting_v1',
                                                '210_Days_After_Planting_v1','240_Days_After_Planting_v1',
                                                '270_Days_After_Planting_v1','At_Harvest_v1'],
                                            index = ['Demo_ID'], aggfunc=np.sum)

banana_fl4= pd.DataFrame(banana_fl3.to_records())
banana_fl4.head()

,Demo_ID,150_Days_After_Planting_v1,180_Days_After_Planting_v1,210_Days_After_Planting_v1,240_Days_After_Planting_v1,270_Days_After_Planting_v1,At_Harvest_v1
0,97634298419,0,0,0,0,0,1.0


# Wheat Observations

In [48]:
path =r'D:\Bizom\Bizom Analysis\Demo Analysis\DemoFY21-22\Crop_Observations\wheat'
wh_ob = pd.concat(map(pd.read_excel, glob.glob(os.path.join(path, "*.xlsx"))),sort=False)

cleaning (wh_ob)
observations(wh_ob)
droptest(wh_ob)

wh_f = wheat_demo.merge(wh_ob,left_on= 'con', right_on = 'DemoID_OutletID')
wh_f.shape

(2, 59)

In [49]:
wh_ob_nm = wh_ob.merge(wh_f,left_on= 'DemoID_OutletID', right_on = 'DemoID_OutletID', how='outer',indicator=True)
wh_ob_nm=wh_ob_nm[wh_ob_nm['_merge']=='left_only']
wh_ob_nm.shape

(89, 84)

In [50]:
wh_f['con_demo_ID_Observation'] = wh_f['Demo_Id']+ wh_f['Demo_Crop___Wheat']
wh_fl = wh_f.drop_duplicates(subset ="con_demo_ID_Observation", keep='last')

wh_fl1= pd.pivot_table(wh_fl,values ='Name', columns ='Demo_Crop___Wheat',
                          index = ['Demo_ID'],  aggfunc={'Name':'count'})

wh_fl2 = pd.DataFrame(wh_fl1.to_records())

wh_fl2.fillna(0, inplace = True)
cleaning(wh_fl2)
wh_fl2.columns

Index(['Demo_ID', '60_Days_After_Planting', 'At_Harvest'], dtype='object')

In [58]:
wh_fl2['60_Days_After_Planting_v1'] = wh_fl2.apply(lambda row: 1.0 if ((row['60_Days_After_Planting']) == 1) and
                                        ((row['At_Harvest']) < 1) else 0 ,axis=1) 

wh_fl2['At_Harvest_v1'] = wh_fl2.apply(lambda row: 1.0 if ((row['60_Days_After_Planting']) == 1) and
                                        ((row['At_Harvest']) ==1) else 0 ,axis=1) 


wh_fl3=  pd.pivot_table(wh_fl2,values =['60_Days_After_Planting_v1','At_Harvest_v1'],index = ['Demo_ID'], aggfunc=np.sum)
wh_fl4= pd.DataFrame(wh_fl3.to_records())
wh_fl4.head()

,Demo_ID,60_Days_After_Planting_v1,At_Harvest_v1
0,97634298421,0,1.0


# Potato Observations

In [52]:
path =r'D:\Bizom\Bizom Analysis\Demo Analysis\DemoFY21-22\Crop_Observations\potato'
pot_ob = pd.concat(map(pd.read_excel, glob.glob(os.path.join(path, "*.xlsx"))),sort=False)

cleaning (pot_ob)
observations(pot_ob)
droptest(pot_ob)

pot_f = pot_demo.merge(pot_ob,left_on= 'con', right_on = 'DemoID_OutletID')
pot_f.shape

(4, 60)

In [53]:
pot_ob_nm = pot_ob.merge(pot_f,left_on= 'DemoID_OutletID', right_on = 'DemoID_OutletID', how='outer',indicator=True)
pot_ob_nm=pot_ob_nm[pot_ob_nm['_merge']=='left_only']
pot_ob_nm.shape

(270, 86)

In [54]:
pot_f['con_demo_ID_Observation'] = pot_f['Demo_Id']+ pot_f['Observation_No_Potato']
pot_fl = pot_f.drop_duplicates(subset ="con_demo_ID_Observation", keep='last')

pot_fl1= pd.pivot_table(pot_fl,values ='Name', columns ='Observation_No_Potato',
                          index = ['Demo_ID'],  aggfunc={'Name':'count'})

pot_fl2 = pd.DataFrame(pot_fl1.to_records())

pot_fl2.fillna(0, inplace = True)
cleaning(pot_fl2)
pot_fl2.columns

Index(['Demo_ID', '1st', 'At_Harvest'], dtype='object')

In [157]:
pot_fl2['1st_v1'] = pot_fl2.apply(lambda row: 1.0 if ((row['1st']) == 1) and
                                        ((row['At_Harvest']) < 1) else 0 ,axis=1) 

pot_fl2['At_Harvest_v1'] = pot_fl2.apply(lambda row: 1.0 if ((row['1st']) == 1) and
                                        ((row['At_Harvest']) ==1) else 0 ,axis=1) 


pot_fl3=  pd.pivot_table(pot_fl2,values =['1st_v1','At_Harvest_v1'],index = ['Demo_ID'], aggfunc=np.sum)
pot_fl4 = pd.DataFrame(pot_fl3.to_records())
pot_fl4.head()

,Demo_ID,1st_v1,At_Harvest_v1
0,97634298418,0,1.0


# DSM

In [ ]:
path =r'D:\Bizom\Bizom Analysis\Demo Analysis\DemoFY21-22\DSM'
dsm = pd.concat(map(pd.read_excel, glob.glob(os.path.join(path, "*.xlsx"))),sort=False)

cleaning (dsm)
droptest(dsm)

dsm[['Demo_ID']] = dsm[['Demo_ID']].fillna(value=0)
dsm[['Demo_ID']] = dsm[['Demo_ID']].astype(int)
dsm[['Demo_ID']] = dsm[['Demo_ID']].astype(str)
dsm['Subdealers_Dealers_Farmers_Villages_Id'] = dsm['Subdealers_Dealers_Farmers_Villages_Id'].astype(str)
dsm['DemoID_OutletID'] = dsm['Demo_ID'] + dsm['Subdealers_Dealers_Farmers_Villages_Id']
dsm['DemoID_OutletID'] = dsm['DemoID_OutletID'].astype(str)

dsm_f = demo1.merge(dsm, left_on= 'con', right_on = 'DemoID_OutletID')
dsm_f.shape

In [ ]:
dsm_nm = dsm.merge(dsm_f,left_on= 'DemoID_OutletID', right_on = 'DemoID_OutletID', how='outer',indicator=True)
dsm_nm=dsm_nm[dsm_nm['_merge']=='left_only']
dsm_nm.shape

In [142]:
dsm1 = pd.pivot_table(dsm_f,values ='User_Name', index = ['Demo_Id'],aggfunc={'User_Name':'count'})

dsm2= pd.DataFrame(dsm1.to_records())
dsm2.rename(columns={'Demo_Id': 'Demo_ID', 'User_Name': 'DSM'}, inplace=True)

dsm2.head()

,Demo_ID,DSM
0,1006117,2
1,1006354,2
2,1006403,2
3,1007799,1
4,1008053,2


# All Observation and DSM Combined

 Exluding Potato and Banana

In [158]:
ob_stg= pd.concat([cot_fl4,oni_fl4,tom_fl4, soy_fl4, gnut_fl4, scane_fl4, paddy_fl4, maize_fl4, must_fl4,wh_fl4, 
                   banana_fl4, pot_fl4])
ob_stg.head()

,Demo_ID,120_Days_After_Planting_v1,60_Days_After_Planting_v1,90_Days_After_Planting_v1,At_Harvest_v1,30_Days_After_Planting_v1,150_Days_After_Planting_v1,180_Days_After_Planting_v1,210_Days_After_Planting_v1,240_Days_After_Planting_v1,270_Days_After_Planting_v1,1st_v1
0,1007965,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1008101,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1008150,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1008212,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1008503,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [159]:
ob_stg1 = ob_stg.merge(dsm2,left_on= 'Demo_ID', right_on = 'Demo_ID', how = 'outer')

print ('df size is', ob_stg1.shape)
ob_stg1.head()

df size is (1354, 13)


,Demo_ID,120_Days_After_Planting_v1,60_Days_After_Planting_v1,90_Days_After_Planting_v1,At_Harvest_v1,30_Days_After_Planting_v1,150_Days_After_Planting_v1,180_Days_After_Planting_v1,210_Days_After_Planting_v1,240_Days_After_Planting_v1,270_Days_After_Planting_v1,1st_v1,DSM
0,1007965,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1008101,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1008150,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1008212,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,1008503,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0


In [184]:
ob_stg1 = ob_stg1 [['Demo_ID','DSM','30_Days_After_Planting_v1','60_Days_After_Planting_v1','90_Days_After_Planting_v1',
                  '120_Days_After_Planting_v1', '150_Days_After_Planting_v1', 'At_Harvest_v1','1st_v1']]

print ('The size of dataframe is' ,ob_stg1.shape)
ob_stg1.head()

The size of dataframe is (1354, 9)


,Demo_ID,DSM,30_Days_After_Planting_v1,60_Days_After_Planting_v1,90_Days_After_Planting_v1,120_Days_After_Planting_v1,150_Days_After_Planting_v1,At_Harvest_v1,1st_v1
0,1007965,NaN,NaN,0.0,0.0,0.0,NaN,0.0,NaN
1,1008101,NaN,NaN,1.0,0.0,0.0,NaN,0.0,NaN
2,1008150,NaN,NaN,1.0,0.0,0.0,NaN,0.0,NaN
3,1008212,1.0,NaN,1.0,0.0,0.0,NaN,0.0,NaN
4,1008503,2.0,NaN,1.0,0.0,0.0,NaN,0.0,NaN


In [177]:
#Drop test demo IDs

demo1.drop(demo1[demo1.Demo_Id.isin(['9763429841', '97634298411', '97634298412', '97634298413', '97634298414', '97634298415',
                           '97634298416', '97634298417', '97634298418', '97634298419', 
                            '97634298420', '97634298421'])].index, inplace= True)

Adding summarize dataframe Demo Dataframe.

In [178]:
demo2 = demo1.merge(ob_stg1, left_on= 'Demo_Id', right_on = 'Demo_ID', how = 'left')

demo2.shape

(8883, 47)

In [179]:
quad =pd.read_excel (r'D:\Bizom\Bizom Analysis\PowerBi\Masters\Dist_q.xlsx')
cleaning (quad)


In [180]:
quad1 = quad[['SFO_Dist', 'Quadrant']].copy()

In [181]:
demo3 = demo2.merge(quad1, left_on= 'District', right_on = 'SFO_Dist', how = 'left')

demo3.shape

(8883, 49)

In [182]:
demo2.to_excel('Demo_Ob_DSM_Final.xlsx', index= False)

# Observations which couldn't match because Crop, Demo ID, Outlet ID

In [153]:
#Function to download all dfs in single excel
def dfs_tabs(df_list, sheet_list, file_name):
    writer = pd.ExcelWriter(file_name,engine='xlsxwriter')   
    for dataframe, sheet in zip(df_list, sheet_list):
        dataframe.to_excel(writer, sheet_name=sheet, startrow=0 , startcol=0)   
    writer.save()

In [174]:
# list of dataframes and sheet names
dfs = [cot_ob_nm, oni_ob_nm,tom_ob_nm,soy_ob_nm, gnut_ob_nm,scane_ob_nm, paddy_ob_nm, maize_ob_nm, must_ob_nm, 
       wh_ob_nm, banana_ob_nm, pot_ob_nm, dsm_nm ]
sheets = ['Cotton','Onion', 'Tomato', 'Soybean', 'Groundnut', 'Sugarcane', 'Paddy', 'Maize', 'Mustard', 
          'Wheat', 'Banana','Potato','DSM']
# run function
dfs_tabs(dfs, sheets, 'Ob_Issues.xlsx')
